In [26]:
import pandas as pd
import re
import string

In [27]:
data = pd.read_csv('datafull.csv')
data.head()

,text
0,Mending plonga plong Tinimbang pinter go minteri
1,Hidup amin Raiso bapa repormasi
2,Mantaappps... Btlllll pak...
3,Yang menghina itu lambemu min
4,Amin rais manusia paling redah pikiranya..peng...


In [28]:
data.isnull().sum()

text    0
dtype: int64

In [29]:
data.duplicated().sum()

343

In [30]:
data.drop_duplicates(inplace=True)

In [31]:
data.duplicated().sum()

0

In [32]:
from collections import Counter
cnt = Counter()
for text in data['text'].values:
    for word in text.split():
        cnt[word] += 1
cnt.most_common(1000)
temp = pd.DataFrame(cnt.most_common(1000))
temp.columns=['word', 'count']
temp


,word,count
0,yg,3856
1,di,2670
2,amin,2662
3,Amin,2565
4,orang,2376
...,...,...
995,Matamu,28
996,"presiden,",28
997,"baik,",28
998,planga,28


In [33]:
data = data[data['text'].notna()]
data = data[data['text'].str.strip() != '']
data = data[data['text'].str.contains(r'[a-zA-Z]{2,}', na=False)]
data

,text
0,Mending plonga plong Tinimbang pinter go minteri
1,Hidup amin Raiso bapa repormasi
2,Mantaappps... Btlllll pak...
3,Yang menghina itu lambemu min
4,Amin rais manusia paling redah pikiranya..peng...
...,...
19342,@kompascom Tapi aiman bertanya ke pengacaranya...
19343,@muannas_alaidid Yg jelas omongannya Jokowi pa...
19344,@CakKhum depresi si om Jokowi
19345,@TriSartopo @ch_chotimah2 @jokowi Mana berani ...


In [34]:
# Fungsi untuk mendeteksi kata dengan perulangan huruf ganda (2+ kali berulang)
def detect_repeated_chars(text):
    pattern = r'\b\w*([a-zA-Z])\1{1,}\w*\b'
    words_with_repetition = []
    words = text.split()
    for word in words:
        clean_word = re.sub(r'[^\w]', '', word.lower())
        if re.search(r'([a-zA-Z])\1{1,}', clean_word):
            words_with_repetition.append(word)

    return words_with_repetition

repeated_words_list = []
for text in data['text'].values:
    words = detect_repeated_chars(str(text))
    repeated_words_list.extend(words)
print(f"Total kata dengan perulangan ditemukan: {len(repeated_words_list)}")
print(f"Kata unik dengan perulangan: {len(set(repeated_words_list))}")


Total kata dengan perulangan ditemukan: 9653
Kata unik dengan perulangan: 5569


In [37]:
from collections import Counter

repeated_counter = Counter(repeated_words_list)
repeated_df = pd.DataFrame(repeated_counter.most_common(100), columns=['kata_berulang', 'frekuensi'])

print("100 kata dengan perulangan huruf terbanyak:")
repeated_df.head(100)

100 kata dengan perulangan huruf terbanyak:


,kata_berulang,frekuensi
0,Allah,213
1,nggak,174
2,saat,115
3,tinggal,101
4,ngga,85
...,...,...
95,jelekkan,10
96,Aminn,10
97,tanggung,10
98,mbahh,10


In [38]:
kata_baku_berulang_final = {
    "allah", "nggak", "saat", "tinggal", "ngga", "alloh", "bukannya", "maaf", "uud", "tinggi", "omongannya", "nunggu", "tunggu",
    "sesungguhnya", "hingga", "ucapannya", "dajjal", "astaghfirullah", "sehingga", "menjelekkan", "meninggal", "sll", "menunjukkan", "panggung", "kerjaan",
    "kenyataan", "sungguh", "bangga", "panggil", "muhammadiyah", "ttp", "nggk", "kekuasaan", "menggonggong", "sllu", "melanggar", "cangkemmu", "kanggo", "menunggu", "dipanggil", "pertanggung", "menggulingkan", "pikirannya", "perkataan", "menganggap", "suul", "keadaan", "saatnya", "muhammad", "engga", "anggota", "kelakukannya", "bloon", "dianggap", "kerjaannya", "manfaatnya", "dll", "diindonesia", "jelekkan", "tanggung"
}

def cleaningText(text, exceptions):
  text = text.lower()
  text = re.sub(r'<br\s*/?>', ' ', text)
  text = re.sub(r'http\S+|www\S+|<a.*?>|</a>', '', text)
  text = re.sub(r'@\w+|#\w+', '', text)
  text = re.sub(r'[^\x00-\x7F]+', ' ', text)
  text = re.sub(r'\d+', '', text)
  text = re.sub(f"[{re.escape(string.punctuation)}]", ' ', text)
  text = ' '.join(text.split())
  tokens = text.split()
  normalized_tokens = []
  for token in tokens:
    if token in exceptions:
      normalized_tokens.append(token)
    else:
      token = re.sub(r'(.)\1+', r'\1', token)
      normalized_tokens.append(token)
  return ' '.join(normalized_tokens)

data['Clean'] = data['text'].apply(lambda x: cleaningText(x, kata_baku_berulang_final))
data

,text,Clean
0,Mending plonga plong Tinimbang pinter go minteri,mending plonga plong tinimbang pinter go minteri
1,Hidup amin Raiso bapa repormasi,hidup amin raiso bapa repormasi
2,Mantaappps... Btlllll pak...,mantaps btl pak
3,Yang menghina itu lambemu min,yang menghina itu lambemu min
4,Amin rais manusia paling redah pikiranya..peng...,amin rais manusia paling redah pikiranya penga...
...,...,...
19342,@kompascom Tapi aiman bertanya ke pengacaranya...,tapi aiman bertanya ke pengacaranya ijazah yan...
19343,@muannas_alaidid Yg jelas omongannya Jokowi pa...,yg jelas omongannya jokowi palsu titik
19344,@CakKhum depresi si om Jokowi,depresi si om jokowi
19345,@TriSartopo @ch_chotimah2 @jokowi Mana berani ...,mana berani tatut sama ibu hm maklum masih cur...


In [40]:
data = data[data['Clean'].notna()]
data = data[data['Clean'].str.strip() != '']
data = data[data['Clean'].str.contains(r'[a-zA-Z]{2,}', na=False)]
data

,text,Clean
0,Mending plonga plong Tinimbang pinter go minteri,mending plonga plong tinimbang pinter go minteri
1,Hidup amin Raiso bapa repormasi,hidup amin raiso bapa repormasi
2,Mantaappps... Btlllll pak...,mantaps btl pak
3,Yang menghina itu lambemu min,yang menghina itu lambemu min
4,Amin rais manusia paling redah pikiranya..peng...,amin rais manusia paling redah pikiranya penga...
...,...,...
19342,@kompascom Tapi aiman bertanya ke pengacaranya...,tapi aiman bertanya ke pengacaranya ijazah yan...
19343,@muannas_alaidid Yg jelas omongannya Jokowi pa...,yg jelas omongannya jokowi palsu titik
19344,@CakKhum depresi si om Jokowi,depresi si om jokowi
19345,@TriSartopo @ch_chotimah2 @jokowi Mana berani ...,mana berani tatut sama ibu hm maklum masih cur...


In [41]:
from collections import Counter
cnt = Counter()
for text in data['Clean'].values:
    for word in text.split():
        cnt[word] += 1
cnt.most_common(1000)
temp = pd.DataFrame(cnt.most_common(1000))
temp.columns=['word', 'count']
temp

,word,count
0,amin,6626
1,yg,4374
2,rais,4037
3,orang,3249
4,ini,3185
...,...,...
995,terlihat,30
996,nilai,30
997,terwujud,30
998,suci,29


In [44]:
norm = {"amin":"", "yg":"yang","rais" : "", "mbah":"kakek", "sengkuni":"licik", "gak":"tidak","gk":"tidak","amien":"", "tobat":"taubat", "sdh":"sudah",
        "ga":"tidak","quot":"kutipan","org":"orang","tdk":"tidak","mu":"kamu","wes":"sudah","wong":"orang","tak":"tidak","mpr":"","gusdur":"","allah":"",
       "lah":"","tau":"tahu","dah":"sudah","bpk":"bapak","lu":"kamu","opo":"apa","jd":"jadi","aki":"kakek","tengil":"menyebalkan","lo":"kamu",
       "tp":"tapi","wis":"sudah","klo":"kalau","to":"","tuwek":"tua","yo":"iya","goblok":"bodoh","d":"","plongo":"bingung","kalo":"kalau","ora":"tidak",
       "g":"tidak","iki":"ini","gus":"","dur":"","mbok":"ibu","pk":"bapak","ra":"tidak","pa":"bapak","plonga":"bingung","iblis":"jahat",
       "nggak":"tidak","bener":"benar","ki":"ini","jgn":"jangan","udh":"sudah","ae":"aja","ko":"kok","dr":"dari","pikun":"lupa","p":"",
       "ni":"ini","km":"kamu","mbh":"kakek","sampean":"kamu","is":"","ngaca":"kaca","asu":"anjing","dgn":"dengan","sih":"","men":"","sing":"yang",
       "wae":"saja","jdi":"jadi","tuek":"tua","pinter":"pintar","rakus":"serakah","amp":"","alloh":"","dg":"dengan","gitu":"begitu","kek":"seperti",
       "bacot":"berisik","inilah":"ini lah","se":"","kowe":"kamu","bin":"","dirimu":"diri kamu","inget":"ingat","pret":"bohong","istighfar":"",
       "raimu":"sial","gini":"begini","modar":"meninggal","prabowo":"","sepuh":"tua","e":"","banget":"sangat","islam":"","waras":"sehat",
       "koyo":"seperti","tuo":"tua","lg":"lagi","mulutmu":"mulut kamu","krn":"karena","dn":"dan","jg":"juga","nih":"ini","cangkem":"mulut",
       "tu":"itu","karna":"karena","iku":"itu","uda":"sudah","prof":"profesor","dadi":"jadi","glandangan":"gelandangan","setan":"jahat","eling":"ingat",
       "kmu":"kamu","edan":"gila","cangkeme":"mulut","sy":"saya","n":"","istigfar":"","cangkemu":"mulut","utk":"untuk","koe":"kamu","blm":"belum",
       "klu":"kalau","seng":"yang","joko":"","ngga":"tidak","nyinyir":"ngomong","msh":"masih","liat":"lihat","sm":"sama","odgj":"gila","mulyono":"","jokowi":""
       }

def normalisasi(text):
    for word, replacement in norm.items():
        text = re.sub(r'\b' + word + r'\b', replacement, text)
    return text

data['normalisasi'] = data['Clean'].apply(normalisasi)
data

,text,Clean,normalisasi
0,Mending plonga plong Tinimbang pinter go minteri,mending plonga plong tinimbang pinter go minteri,mending bingung plong tinimbang pintar go minteri
1,Hidup amin Raiso bapa repormasi,hidup amin raiso bapa repormasi,hidup raiso bapa repormasi
2,Mantaappps... Btlllll pak...,mantaps btl pak,mantaps btl pak
3,Yang menghina itu lambemu min,yang menghina itu lambemu min,yang menghina itu lambemu min
4,Amin rais manusia paling redah pikiranya..peng...,amin rais manusia paling redah pikiranya penga...,manusia paling redah pikiranya pengabdi hawa...
...,...,...,...
19342,@kompascom Tapi aiman bertanya ke pengacaranya...,tapi aiman bertanya ke pengacaranya ijazah yan...,tapi aiman bertanya ke pengacaranya ijazah yan...
19343,@muannas_alaidid Yg jelas omongannya Jokowi pa...,yg jelas omongannya jokowi palsu titik,yang jelas omongannya palsu titik
19344,@CakKhum depresi si om Jokowi,depresi si om jokowi,depresi si om
19345,@TriSartopo @ch_chotimah2 @jokowi Mana berani ...,mana berani tatut sama ibu hm maklum masih cur...,mana berani tatut sama ibu hm maklum masih cur...


In [45]:
from collections import Counter
cnt = Counter()
for text in data['normalisasi'].values:
    for word in text.split():
        cnt[word] += 1
cnt.most_common(1000)
temp = pd.DataFrame(cnt.most_common(1000))
temp.columns=['word', 'count']
temp

,word,count
0,yang,6114
1,tidak,5274
2,orang,4291
3,ini,4073
4,kakek,3921
...,...,...
995,patut,25
996,sambil,25
997,ttp,25
998,peci,25
